# 4. Wie  entwickelt  sich  der  sportliche  Erfolg  mit  dem  Alter?  Gilt  das  für  die  Spielstatistiken gleichermaßen wie für die Platzierung auf der Rangliste?

In [62]:
import numpy as np
import pandas as pd

In [63]:
matches = pd.read_csv('../data/raw/atp_matches_till_2022.csv')
players = pd.read_csv('../data/raw/atp_players_till_2022.csv')
rankings = pd.read_csv('../data/raw/atp_rankings_till_2022.csv')

In [64]:
players
# show all duplicates in player_id

,player_id,name_first,name_last,hand,dob,ioc,height,wikidata_id
0,100001,Gardnar,Mulloy,R,19131122.0,USA,185.0,Q54544
1,100002,Pancho,Segura,R,19210620.0,ECU,168.0,Q54581
2,100003,Frank,Sedgman,R,19271002.0,AUS,180.0,Q962049
3,100004,Giuseppe,Merlo,R,19271011.0,ITA,NaN,Q1258752
4,100005,Richard,Gonzalez,R,19280509.0,USA,188.0,Q53554
...,...,...,...,...,...,...,...,...
58682,212032,Abhishek,Bastola,U,NaN,NPL,NaN,NaN
58683,212033,Cameron,Muller,U,NaN,USA,NaN,NaN
58684,212034,Ian,Bracks,U,NaN,USA,NaN,NaN
58685,212035,Alexander,Visser,U,NaN,USA,NaN,NaN


In [65]:
# take rankings and add dob from players (join on player_id in players and player in rankings)
rankings = rankings.merge(players[['player_id', 'dob']], how='left', left_on='player', right_on='player_id')

In [66]:
rankings

,ranking_date,rank,player,points,player_id,dob
0,20100104,1,103819,10550.0,103819,19810808.0
1,20100104,2,104745,9205.0,104745,19860603.0
2,20100104,3,104925,8310.0,104925,19870522.0
3,20100104,4,104918,7030.0,104918,19870515.0
4,20100104,5,105223,6785.0,105223,19880923.0
...,...,...,...,...,...,...
3113256,20091228,1805,111421,1.0,111421,19910228.0
3113257,20091228,1809,105648,1.0,105648,19901008.0
3113258,20091228,1809,109144,1.0,109144,19901107.0
3113259,20091228,1811,105675,1.0,105675,19901205.0


In [67]:
# convert dob to datetime in format yyyymmdd
rankings['dob'] = pd.to_datetime(rankings['dob'], format='%Y%m%d')
rankings = rankings.rename(columns={'dob': 'birth_date'})
rankings.sort_values(by='player_id')

,ranking_date,rank,player,points,player_id,birth_date
1169016,19770704,366,100001,NaN,100001,1913-11-22
1169419,19780102,414,100001,NaN,100001,1913-11-22
1169888,19780116,397,100001,NaN,100001,1913-11-22
1166893,19750505,399,100002,NaN,100002,1921-06-20
1167847,19751215,373,100002,NaN,100002,1921-06-20
...,...,...,...,...,...,...
1157915,20221219,1470,211758,2.0,211758,NaT
1159907,20221226,1469,211758,2.0,211758,NaT
1158120,20221219,1645,211763,1.0,211763,NaT
1160117,20221226,1650,211763,1.0,211763,NaT


In [68]:
rankings.dropna(inplace=True)

In [69]:
rankings[rankings['player'] == 102093].sort_values(by='ranking_date')

,ranking_date,rank,player,points,player_id,birth_date
1467471,19900101,722,102093,8.0,102093,2003-09-30
1468740,19900108,716,102093,8.0,102093,2003-09-30
1470028,19900115,713,102093,8.0,102093,2003-09-30
1472608,19900129,719,102093,8.0,102093,2003-09-30
1475173,19900212,716,102093,8.0,102093,2003-09-30
...,...,...,...,...,...,...
1152804,20221128,463,102093,87.0,102093,2003-09-30
1154796,20221205,461,102093,87.0,102093,2003-09-30
1156788,20221212,458,102093,87.0,102093,2003-09-30
1158777,20221219,455,102093,87.0,102093,2003-09-30


In [70]:
from datetime import datetime

# Convert the date strings to datetime objects
rankings['ranking_date'] = pd.to_datetime(rankings['ranking_date'], format='%Y%m%d')
rankings['birth_date'] = pd.to_datetime(rankings['birth_date'])

# Calculate the age of the players at the time of the ranking
rankings['age'] = rankings['ranking_date'].dt.year - rankings['birth_date'].dt.year
rankings.loc[rankings['ranking_date'].dt.month < rankings['birth_date'].dt.month, 'age'] -= 1
rankings.loc[(rankings['ranking_date'].dt.month == rankings['birth_date'].dt.month) & 
             (rankings['ranking_date'].dt.day < rankings['birth_date'].dt.day), 'age'] -= 1


In [71]:
# drop all rows where age is negative
rankings = rankings[rankings['age'] >= 0]

In [72]:
import plotly.graph_objects as go

age_group_stats = rankings.groupby('age').agg({'rank': 'mean', 'points': 'mean'}).reset_index()

# Erstellung des ersten Linienplots für den Durchschnittsrang
fig = go.Figure()
fig.add_trace(go.Scatter(x=age_group_stats['age'], y=age_group_stats['rank'], mode='lines', name='Durchschnittlicher Rang'))

# Erstellung des zweiten Linienplots für die Durchschnittspunkte
fig.add_trace(go.Scatter(x=age_group_stats['age'], y=age_group_stats['points'], mode='lines', name='Durchschnittliche Punkte', yaxis="y2"))

# Hinzufügen von Achsentiteln und Titel des Diagramms
fig.update_layout(
    title='Durchschnittlicher Rang und Punkte nach Alter',
    xaxis_title='Alter',
    yaxis_title='Durchschnittlicher Rang',
    yaxis2=dict(
        title='Durchschnittliche Punkte',
        overlaying='y',
        side='right'
    )
)

# Anzeigen des Diagramms
fig.show()

# Korrelationsanalyse

In [73]:
correlation_rank_age = rankings['age'].corr(rankings['rank'])
correlation_points_age = rankings['age'].corr(rankings['points'])


In [74]:
print(f"Es gab eine negative Korrelation von {correlation_rank_age.round(4)} zwischen dem Alter und dem Rang, was darauf hindeutet, dass jüngere Spieler tendenziell höhere Ranglistenpositionen einnehmen.\n"\
      f"Die Korrelation zwischen Alter und Punkten war positiv {correlation_points_age.round(4)}, was darauf hindeutet, dass ältere Spieler tendenziell mehr Punkte haben.")

Es gab eine negative Korrelation von -0.2726 zwischen dem Alter und dem Rang, was darauf hindeutet, dass jüngere Spieler tendenziell höhere Ranglistenpositionen einnehmen.
Die Korrelation zwischen Alter und Punkten war positiv 0.1608, was darauf hindeutet, dass ältere Spieler tendenziell mehr Punkte haben.


# Lineare Regression

In [75]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Lineare Regression für Alter vs. Rang
regressor_age_rank = LinearRegression()
regressor_age_rank.fit(rankings[['age']], rankings['rank'])
predicted_rank = regressor_age_rank.predict(rankings[['age']])
r2_rank = r2_score(rankings['rank'], predicted_rank)

# Lineare Regression für Alter vs. Punkte
regressor_age_points = LinearRegression()
regressor_age_points.fit(rankings[['age']], rankings['points'])
predicted_points = regressor_age_points.predict(rankings[['age']])
r2_points = r2_score(rankings['points'], predicted_points)

coef_rank = regressor_age_rank.coef_[0]
coef_points = regressor_age_points.coef_[0]


In [76]:
coef_rank, coef_points

(-34.05136428453809, 16.46482825994343)

In [77]:
print(f"Für Alter vs. Rang: Der Koeffizient betrug {coef_rank}, was bedeutet, dass der Rang im Durchschnitt um {coef_rank.round()} Positionen sinkt, je älter ein Spieler wird. \nDas Bestimmtheitsmaß (R²) war {r2_rank}, was eine moderate Vorhersagekraft anzeigt.\n\n"\
    f"Für Alter vs. Punkte: Der Koeffizient betrug {coef_points}, was bedeutet, dass die Punktzahl im Durchschnitt um etwa {coef_points.round()} Punkte steigt, je älter ein Spieler wird. \nDas Bestimmtheitsmaß (R²) war jedoch mit {r2_points} sehr gering.")

Für Alter vs. Rang: Der Koeffizient betrug -34.05136428453809, was bedeutet, dass der Rang im Durchschnitt um -34.0 Positionen sinkt, je älter ein Spieler wird. 
Das Bestimmtheitsmaß (R²) war 0.07433519443833692, was eine moderate Vorhersagekraft anzeigt.

Für Alter vs. Punkte: Der Koeffizient betrug 16.46482825994343, was bedeutet, dass die Punktzahl im Durchschnitt um etwa 16.0 Punkte steigt, je älter ein Spieler wird. 
Das Bestimmtheitsmaß (R²) war jedoch mit 0.025860793594407627 sehr gering.


# Untersuchung auf stochastische Unabhängigkeit: 
Mithilfe eines Chi-Quadrat-Tests können wir prüfen, ob Alter und Ranglistenposition unabhängig voneinander sind

In [78]:
from scipy.stats import chi2_contingency

# Erstellung der Kontingenztafel
contingency_table = pd.crosstab(rankings['age'], rankings['rank'])

# Durchführung des Chi-Quadrat-Tests
chi2, p, dof, expected = chi2_contingency(contingency_table)

chi2, p

(617709.5824034236, 0.0)

Der Chi-Quadrat-Test ergab einen p-Wert von 0.0. (<0.05) Dies bedeutet, dass wir die Nullhypothese der Unabhängigkeit zwischen Alter und Rangliste ablehnen können. 
Der extrem niedrige p-Wert zeigt, dass es eine sehr signifikante Beziehung zwischen dem Alter der Spieler und ihrer Position in der Rangliste gibt. Dies bestätigt, dass der Zusammenhang, den wir in den vorherigen Analysen beobachtet haben, nicht zufällig ist, sondern systematisch.